In [1]:
import numpy as np, random, operator,copy
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
class Edge :
    def __init__(self, startNode, endNode, domain, weight, visitedNode) :
        self.startNode = startNode
        self.endNode = endNode
        self.domain = domain
        self.weight = weight
        self.visitedNode = visitedNode

    def __add__(self,other): 
        assert self.endNode == other.startNode
        assert self.domain == other.domain
        visitedNode = []
        for node in self.visitedNode :
            visitedNode.append(node)
        for node in other.visitedNode :
            if node not in self.visitedNode :
                visitedNode.append(node)
        weight = self.weight + other.weight
        return Edge (self.startNode, other.endNode, self.domain, weight, visitedNode)

    def __gt__ (self, other) :
        if (self.weight > other.weight) :
            return True
        else :
            if self.weight == other.weight and len(self.visitedNode) > len(other.visitedNode) :
                return True
            else :
                return False

In [3]:
file = open("set1/idpc_15x15x3375.idpc","r")
lines = file.readlines()
tmp = lines[0].split()
numNodes = int(tmp[0])
numDomains = int(tmp[1])
weight_matrix = np.ones([numNodes, numNodes, numDomains]) * -1
tmp = lines[1].split()
start = int(tmp[0]) - 1
end = int(tmp[1]) - 1
adj1 = []
node_to_domain = []
for i in range(numNodes) :
    tmp = []
    node_to_domain.append(tmp)
for i in range(numNodes) :
    adj_temp = []
    for j in range(numNodes) :
        domain_list = [1] * numDomains
        adj_temp.append(domain_list)
    adj1.append(adj_temp)

adj2 = []
for i in range(numDomains) :
    node_list = []
    for i in range(numNodes) :
        tmp = []
        node_list.append(tmp)
    adj2.append(node_list)

for i in range(2,len(lines)) :
    tmp = lines[i].split()
    weight = int(tmp[2])
    domain = int(tmp[3]) - 1
    startNode = int(tmp[0]) - 1
    endNode = int(tmp[1]) - 1

    # weight[startNode, endNode, domain] = weight
    if startNode == end or endNode == start :
        continue
    edge = Edge (startNode, endNode, domain, weight, visitedNode= [])
    adj2[domain][startNode].append(edge)
    if domain not in node_to_domain :
        node_to_domain[startNode].append(domain)

FileNotFoundError: [Errno 2] No such file or directory: 'set1/idpc_15x15x3375.idpc'

In [ ]:
from queue import PriorityQueue
for domain in range(numDomains) :
    adj_domain = adj2[domain]
    for i in range(numNodes) :
        check = [True] * numNodes
        priority_queue = PriorityQueue()
        for edge in adj_domain[i] :
            priority_queue.put(edge)
        if priority_queue.empty() == False :
            edge = priority_queue.get()
            adj1[edge.startNode][ edge.endNode][edge.domain] = edge
        while priority_queue.empty() == False :
            next_Node = edge.endNode
            check[next_Node] = False
            for candidate in adj_domain[next_Node] :
                try :
                    priority_queue.put(edge + candidate)
                except :
                    print("___________________")
                    print(edge.endNode)
                    for c in adj_domain[next_Node] :
                        print(c.startNode)

            edge = priority_queue.get()
            while check[edge.endNode] == False and priority_queue.empty() == False :
                edge = priority_queue.get()
            if check[next_Node] == True :
                adj1[edge.startNode][ edge.endNode][edge.domain] = edge
            

In [ ]:
def intersection (list1, list2) :
    for i in list1 :
        if i in list2 : 
            return True
    return False

In [ ]:
def find_next_domain (priority_list, visited_list, node_to_domain) :
    priority_list_tmp = copy.deepcopy(priority_list)
    for i in visited_list :
        priority_list_tmp[i] = -1
    for i in range(numDomains) :
        if i not in node_to_domain :
            priority_list_tmp[i] = -1
    return np.argmax(priority_list_tmp)

In [ ]:
def find_next_Node (priority_list, visited_list, domain, current_Node) :
    global adj1
    priority_list_tmp = copy.deepcopy(priority_list)
    for i in visited_list :
        priority_list_tmp[i] = -1
    candidate_Node = np.argmax(priority_list_tmp)
    stop = False
    if adj1[current_Node][candidate_Node][domain] == 1 :
        stop = True
    else :
        visited_Node = adj1[current_Node][candidate_Node][domain].visitedNode
        if intersection(visited_list, visited_Node) == True :
            stop = True
    while stop == True :
        priority_list_tmp[candidate_Node] = -1
        if np.sum(priority_list_tmp) < -(numNodes - 1) :
            return None
        candidate_Node = np.argmax(priority_list_tmp)
        stop = False
        if adj1[current_Node][candidate_Node][domain] == 1 :
            stop = True
        else :
            visited_Node = adj1[current_Node][candidate_Node][domain].visitedNode
            if intersection(visited_list, visited_Node) == True :
                stop = True
    return candidate_Node

In [27]:
def fitness (genes) :
    global start, end
    visitedNodes = []
    visitedDomain = []
    genes_domain = genes[0:numDomains]
    genes_node  = genes[numDomains : (numNodes+ numDomains)]
    current_Domain = 0
    current_Node = start
    distance = 0
    residual = 1000000
    while current_Node != end :
        current_Domain = find_next_domain (genes_domain, visitedDomain, node_to_domain[current_Node])
        next_Node = find_next_Node (genes_node, visitedNodes, current_Domain, current_Node)

        if adj1[current_Node][end][current_Domain] != 1 :
            residual = min (residual, distance + adj1[current_Node][end][current_Domain].weight)

        if next_Node == None :
            return residual

        distance += adj1[current_Node][next_Node][current_Domain].weight
        visitedNodes.append(next_Node)
        visitedNodes += adj1[current_Node][next_Node][current_Domain].visitedNode
        visitedDomain.append(current_Domain)
        current_Node = next_Node
    return min(distance, residual)
        

In [28]:
class Individual :
    def __init__ (self, genes) :
        self.genes = genes
        self.fitness = fitness(genes)

In [29]:
class IndClass(Individual):
    def __init__(self, genes, parent= None, dim= None, *args, **kwargs) -> None:
        super().__init__(genes, parent, dim)
        if genes is None:
            self.genes: np.ndarray = np.random.rand(dim)
        self.A = np.ones(len(self.genes))